In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark
from delta import *
import os

In [4]:

# Build Spark session with Delta configurations and Hive support
builder = SparkSession.builder.appName("silvertogold") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.io.native.lib.available", "true") \
    .config("spark.jars.packages", "org.apache.hive:hive-exec:2.3.9")  \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
# Enable Hive support explicitly and get Spark session
spark = configure_spark_with_delta_pip(builder.enableHiveSupport()).getOrCreate()

# Optionally, you can test by running a Hive query
spark.sql("CREATE TABLE IF NOT EXISTS test_hive_table (name STRING, age INT) USING hive")
spark.sql("SELECT * FROM test_hive_table")
spark.sql("select count(*) from silver.location").show()


+--------+
|count(1)|
+--------+
| 1006477|
+--------+



In [5]:
# List of external silver tables
silver_tables = [
    "silver.location",
    "silver.payment",
    "silver.payment_method",
    "silver.payment_status",
    "silver.request",
    "silver.trip",
    "silver.user",
    "silver.vehicle",
    "silver.vehicle_make"
] 
print(silver_tables)

['silver.location', 'silver.payment', 'silver.payment_method', 'silver.payment_status', 'silver.request', 'silver.trip', 'silver.user', 'silver.vehicle', 'silver.vehicle_make']


In [6]:
landing_location = os.path.join(os.getcwd())
bronze_location = os.path.join(landing_location, "bronze")
silver_location = fr"{os.getcwd()}/silver"
gold_location = fr"{os.getcwd()}/gold"
print(silver_tables)
print(os.getcwd())
print(bronze_location)
print(silver_location)
print(gold_location)

['silver.location', 'silver.payment', 'silver.payment_method', 'silver.payment_status', 'silver.request', 'silver.trip', 'silver.user', 'silver.vehicle', 'silver.vehicle_make']
e:\DEPIfINALpROJECT\DATABASEcSV
e:\DEPIfINALpROJECT\DATABASEcSV\bronze
e:\DEPIfINALpROJECT\DATABASEcSV/silver
e:\DEPIfINALpROJECT\DATABASEcSV/gold


In [7]:
# Verify the current schema
print("Current Schema:")
spark.sql("use silver")
spark.sql("SELECT current_database()")

# List all tables in the bronze schema
print("Tables in silver schema:")
spark.sql("SHOW TABLES IN silver").show()
spark.sql("SELECT * FROM silver.location LIMIT 1").show()
spark.sql("SELECT * FROM silver.payment LIMIT 1").show()


# Describe the history of the Delta table
print("Describing history for silver.location:")



Current Schema:
Tables in silver schema:
+---------+--------------+-----------+
|namespace|     tableName|isTemporary|
+---------+--------------+-----------+
|   silver|      location|      false|
|   silver|       payment|      false|
|   silver|payment_method|      false|
|   silver|payment_status|      false|
|   silver|       request|      false|
|   silver|          trip|      false|
|   silver|          user|      false|
|   silver|       vehicle|      false|
|   silver|  vehicle_make|      false|
+---------+--------------+-----------+

+-----------+------------+-----------+-------------------+--------------------+--------------------+
|location_id|   longitude|   latitude|    processing_date|record_modified_date|         record_hash|
+-----------+------------+-----------+-------------------+--------------------+--------------------+
|     865126|-74.00482178|40.73152542|2024-10-13 19:44:50|2024-10-08 09:46:...|000040462abe6e0e6...|
+-----------+------------+-----------+---------

In [8]:

deltaTable = DeltaTable.forPath(spark, "silver\\location")
fullHistoryDF = deltaTable.history()
fullHistoryDF.show(truncate=False)

+-------+-----------------------+------+--------+------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+-----------------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------

In [9]:
dim_date = "gold.DIM_DATES"
dim_user = "gold.DIM_USERS"
dim_vehicle = "gold.DIM_VEHICLES"
dim_location = "gold.DIM_LOCATIONS"
fact_request = "gold.FCT_REQUESTS"

In [10]:
from pyspark.sql.functions import to_date, current_date

# Read data from each table and assign to individual variables
user_df = spark.read.table("silver.user").filter(to_date("processing_date") == current_date())
location_df = spark.read.table("silver.location").filter(to_date("processing_date") == current_date())
payment_df = spark.read.table("silver.payment").filter(to_date("processing_date") == current_date())
paymentmethod_df = spark.read.table("silver.payment_method").filter(to_date("processing_date") == current_date())
paymentstatus_df = spark.read.table("silver.payment_status").filter(to_date("processing_date") == current_date())
request_df = spark.read.table("silver.request").filter(to_date("processing_date") == current_date())
trip_df = spark.read.table("silver.trip").filter(to_date("processing_date") == current_date())
vehicle_df = spark.read.table("silver.vehicle").filter(to_date("processing_date") == current_date())
vehiclemake_df = spark.read.table("silver.vehicle_make").filter(to_date("processing_date") == current_date())

# Show each DataFrame separately
user_df.show()
location_df.show()
payment_df.show()
paymentmethod_df.show()
paymentstatus_df.show()
request_df.show()
trip_df.show()
vehicle_df.show()
vehiclemake_df.show()


+-------+---------------+--------------------+--------------+-------------------+--------------------+--------------------+
|user_id|      full_name|               email|  phone_number|    processing_date|record_modified_date|         record_hash|
+-------+---------------+--------------------+--------------+-------------------+--------------------+--------------------+
|  21485| Anthony Morgan|ubradford@example...|(175) 891-4063|2024-10-13 19:44:31|2024-10-13 13:07:...|000082258f3c61f26...|
|1017693|  Jesse Gregory|garyrodriguez@exa...|(806) 231-5009|2024-10-13 19:44:31|2024-10-13 13:07:...|00009d1306f8d0d7e...|
|1174541|  Michael Davis|reyesdaniel@examp...|(724) 952-2855|2024-10-13 19:44:31|2024-10-13 13:34:...|00019ce038354f826...|
| 986574|Haley Alexander|joannejohnson@exa...|(606) 214-3136|2024-10-13 19:44:31|2024-10-13 13:34:...|0001d7e8506472aab...|
|1014405|   Lori Trevino|laurenrobbins@exa...|(393) 778-7835|2024-10-13 19:44:31|2024-10-13 13:34:...|000554497b1fe9ec8...|
| 108642

In [11]:
user_df.orderBy("full_name").show()

+-------+------------+--------------------+--------------+-------------------+--------------------+--------------------+
|user_id|   full_name|               email|  phone_number|    processing_date|record_modified_date|         record_hash|
+-------+------------+--------------------+--------------+-------------------+--------------------+--------------------+
|   NULL|    --------|               -----|   -----------|2024-10-13 19:44:31|2024-10-13 13:07:...|370d9b71060be968d...|
| 768128|Aaron Abbott|angela67@example.com|(453) 832-3608|2024-10-13 19:44:31|2024-10-13 13:34:...|1ec48e73a3376b697...|
| 762535|Aaron Abbott|angela67@example.com|(453) 832-3608|2024-10-13 19:44:31|2024-10-13 13:34:...|7c4dfcfb597e8c6e2...|
| 767104|Aaron Abbott|angela67@example.com|(453) 832-3608|2024-10-13 19:44:31|2024-10-13 13:34:...|24d00ea8fc1be35ea...|
| 776168|Aaron Abbott|angela67@example.com|(453) 832-3608|2024-10-13 19:44:31|2024-10-13 13:34:...|408174e83dc4afa4d...|
| 967068|Aaron Acosta|vanessa20@

In [12]:
# Step 1: Join user_df with vehicle_df on user_id (assuming driver_id in vehicle_df relates to user_id)
user_vehicle_df = user_df.join(vehicle_df, user_df["user_id"] == vehicle_df["driver_id"], "left")

# Step 2: Join vehicle_df with vehicle_make_df on make_id to bring in make details
user_vehicle_make_df = user_vehicle_df.join(vehiclemake_df, vehicle_df["make_id"] == vehiclemake_df["make_id"], "left")

In [13]:
dim_user = user_vehicle_make_df.select(
    "user_id",
    "full_name",
    "email",
    "phone_number",
    # Replace NULLs in vehicle columns with 'Passenger' or 'N/A'
    coalesce(user_vehicle_make_df["vehicle_id"], lit("Passenger")).alias("vehicle_id"),
    coalesce(user_vehicle_make_df["make_name"], lit("Passenger")).alias("vehicle_make"),
    coalesce(user_vehicle_make_df["model"], lit("Passenger")).alias("vehicle_model"),
    coalesce(user_vehicle_make_df["year"], lit("Passenger")).alias("vehicle_year"),
    coalesce(user_vehicle_make_df["color"], lit("Passenger")).alias("vehicle_color"),
    coalesce(user_vehicle_make_df["license_plate"], lit("Passenger")).alias("vehicle_license_plate")
)




In [14]:
dim_user.orderBy("full_name").show()

+-------+------------+--------------------+--------------+----------+------------+-------------+------------+-------------+---------------------+
|user_id|   full_name|               email|  phone_number|vehicle_id|vehicle_make|vehicle_model|vehicle_year|vehicle_color|vehicle_license_plate|
+-------+------------+--------------------+--------------+----------+------------+-------------+------------+-------------+---------------------+
|   NULL|    --------|               -----|   -----------| Passenger|   Passenger|    Passenger|   Passenger|    Passenger|            Passenger|
| 762535|Aaron Abbott|angela67@example.com|(453) 832-3608|    168434|     Hyundai|     Santa Fe|        2006|       Silver|              3BW 116|
| 768128|Aaron Abbott|angela67@example.com|(453) 832-3608|    168434|     Hyundai|     Santa Fe|        2006|       Silver|              3BW 116|
| 776168|Aaron Abbott|angela67@example.com|(453) 832-3608|    168434|     Hyundai|     Santa Fe|        2006|       Silver| 

In [15]:

# Define the start and end date
start_date = "2015-01-01"
end_date = "2015-01-31"

# Create a Spark DataFrame with a date range using the sequence function
calendar_df = spark.sql(f"""
    SELECT explode(sequence(to_date('{start_date}'), to_date('{end_date}'), interval 1 day)) as full_date
""")

# Transform the DataFrame to create the calendar dimension
dim_calendar = calendar_df.select(
    date_format("full_date", "yyyyMMdd").alias("date_key"),  # Convert to string format YYYYMMDD
    col("full_date"),
    dayofmonth("full_date").alias("day"),  # Day of the month
    date_format("full_date", "EEEE").alias("day_name"),  # Full name of the day
    dayofweek("full_date").alias("day_of_week"),  # Day of the week (1 = Sunday, 7 = Saturday)
    weekofyear("full_date").alias("week_of_year"),  # Week of the year
    month("full_date").alias("month"),  # Month (1-12)
    date_format("full_date", "MMMM").alias("month_name"),  # Full name of the month
    quarter("full_date").alias("quarter"),  # Quarter (1-4)
    year("full_date").alias("year"),  # Year
    when(dayofweek("full_date").isin([1, 7]), 1).otherwise(0).alias("is_weekend"),  # Weekend flag (1=weekend)
    lit(0).alias("is_holiday"),  # Dummy holiday flag (replace with actual holiday logic)
    month("full_date").alias("fiscal_month"),  # Fiscal month (you can customize this logic)
    quarter("full_date").alias("fiscal_quarter"),  # Fiscal quarter (you can customize this logic)
    year("full_date").alias("fiscal_year")  # Fiscal year
)

dim_calendar.printSchema()
# Show the resulting calendar dimension DataFrame



root
 |-- date_key: string (nullable = false)
 |-- full_date: date (nullable = false)
 |-- day: integer (nullable = false)
 |-- day_name: string (nullable = false)
 |-- day_of_week: integer (nullable = false)
 |-- week_of_year: integer (nullable = false)
 |-- month: integer (nullable = false)
 |-- month_name: string (nullable = false)
 |-- quarter: integer (nullable = false)
 |-- year: integer (nullable = false)
 |-- is_weekend: integer (nullable = false)
 |-- is_holiday: integer (nullable = false)
 |-- fiscal_month: integer (nullable = false)
 |-- fiscal_quarter: integer (nullable = false)
 |-- fiscal_year: integer (nullable = false)



In [16]:
dim_calendar.show(8)

+--------+----------+---+---------+-----------+------------+-----+----------+-------+----+----------+----------+------------+--------------+-----------+
|date_key| full_date|day| day_name|day_of_week|week_of_year|month|month_name|quarter|year|is_weekend|is_holiday|fiscal_month|fiscal_quarter|fiscal_year|
+--------+----------+---+---------+-----------+------------+-----+----------+-------+----+----------+----------+------------+--------------+-----------+
|20150101|2015-01-01|  1| Thursday|          5|           1|    1|   January|      1|2015|         0|         0|           1|             1|       2015|
|20150102|2015-01-02|  2|   Friday|          6|           1|    1|   January|      1|2015|         0|         0|           1|             1|       2015|
|20150103|2015-01-03|  3| Saturday|          7|           1|    1|   January|      1|2015|         1|         0|           1|             1|       2015|
|20150104|2015-01-04|  4|   Sunday|          1|           1|    1|   January|     

In [17]:
paymentmethod_df.show()
paymentstatus_df.show()

+-----------------+-----------+-------------------+--------------------+--------------------+
|payment_method_id|method_name|    processing_date|record_modified_date|         record_hash|
+-----------------+-----------+-------------------+--------------------+--------------------+
|             NULL| ----------|2024-10-13 19:45:10|2024-10-08 09:47:...|e3b0c44298fc1c149...|
|                1|  Apple Pay|2024-10-13 19:45:10|2024-10-08 09:47:...|6b86b273ff34fce19...|
|                2|       Cash|2024-10-13 19:45:10|2024-10-08 09:47:...|d4735e3a265e16eee...|
|                3|Credit Card|2024-10-13 19:45:10|2024-10-08 09:47:...|4e07408562bedb8b6...|
|                4| Debit Card|2024-10-13 19:45:10|2024-10-08 09:47:...|4b227777d4dd1fc61...|
|                5| Google Pay|2024-10-13 19:45:10|2024-10-08 09:47:...|ef2d127de37b942ba...|
|                6|     PayPal|2024-10-13 19:45:10|2024-10-08 09:47:...|e7f6c011776e8db7c...|
+-----------------+-----------+-------------------+---------

In [18]:
# Perform a cross join to get all combinations of PaymentMethod and PaymentStatus
combined_df = paymentmethod_df.crossJoin(paymentstatus_df)

# Add an incremental surrogate key
final_df = combined_df.withColumn("s_payment", monotonically_increasing_id())

# Select only the relevant columns (SurrogateKey, PaymentMethodID, PaymentStatusID)
final_df = final_df.select("s_payment", "payment_method_id", "method_name", "payment_status_id", "status_name")

# Show the resulting DataFrame
final_df.show(30)

+---------+-----------------+-----------+-----------------+-----------+
|s_payment|payment_method_id|method_name|payment_status_id|status_name|
+---------+-----------------+-----------+-----------------+-----------+
|        0|             NULL| ----------|             NULL| ----------|
|        1|             NULL| ----------|                1|  Completed|
|        2|             NULL| ----------|                2|     Failed|
|        3|             NULL| ----------|                3|    Pending|
|        4|             NULL| ----------|                4|   Refunded|
|        5|                1|  Apple Pay|             NULL| ----------|
|        6|                1|  Apple Pay|                1|  Completed|
|        7|                1|  Apple Pay|                2|     Failed|
|        8|                1|  Apple Pay|                3|    Pending|
|        9|                1|  Apple Pay|                4|   Refunded|
|       10|                2|       Cash|             NULL| ----

In [19]:
payment_df.show()

+----------+-----------------+-----------------+-------------------+--------------------+--------------------+
|payment_id|payment_method_id|payment_status_id|    processing_date|record_modified_date|         record_hash|
+----------+-----------------+-----------------+-------------------+--------------------+--------------------+
|      NULL|             NULL|             NULL|2024-10-13 19:45:08|2024-10-08 09:47:...|e3b0c44298fc1c149...|
|         1|                1|                1|2024-10-13 19:45:08|2024-10-08 09:47:...|6b86b273ff34fce19...|
|         6|                6|                4|2024-10-13 19:45:08|2024-10-08 09:47:...|e7f6c011776e8db7c...|
|         3|                3|                1|2024-10-13 19:45:08|2024-10-08 09:47:...|4e07408562bedb8b6...|
|         5|                5|                2|2024-10-13 19:45:08|2024-10-08 09:47:...|ef2d127de37b942ba...|
|         4|                4|                4|2024-10-13 19:45:08|2024-10-08 09:47:...|4b227777d4dd1fc61...|
|

In [20]:
# Perform the join operation
dim_payment = payment_df.alias("p").join(
    combined_df.alias("c"),
    (col("p.payment_method_id") == col("c.payment_method_id")) &
    (col("p.payment_status_id") == col("c.payment_status_id")),
    "inner"  # Use "inner" for only matching records
).select(
    col("p.payment_id").alias("s_payment"),  # Select from payment_df
    col("c.method_name").alias("method"),  # Select from combined_df
    col("c.status_name").alias("status")  # Select from combined_df
)

# Show the resulting DataFrame
dim_payment.show(50)

+---------+-----------+---------+
|s_payment|     method|   status|
+---------+-----------+---------+
|        1|  Apple Pay|Completed|
|        2|       Cash|  Pending|
|        3|Credit Card|Completed|
|        4| Debit Card| Refunded|
|        5| Google Pay|   Failed|
|        6|     PayPal| Refunded|
+---------+-----------+---------+



In [21]:
dim_location = location_df.select("location_id", "latitude", "longitude")
dim_location.show()

+-----------+-----------+------------+
|location_id|   latitude|   longitude|
+-----------+-----------+------------+
|     140808|40.74440384|-73.99198151|
|    1099001|40.72247696|-73.98711395|
|     419142|40.75440979|-73.96898651|
|     405519|40.76442337| -73.9709549|
|     428147|40.75773621|-73.96775818|
|     232211|40.74724197|-73.98554993|
|    1435854|40.76414871|-73.96662903|
|    1024948|40.73995972|-73.98249054|
|    1601613|40.75263977|-73.96591949|
|    1100722|40.77864838| -73.9495163|
|     564879|40.64452744|-73.78232574|
|     963528| 40.6946106|-73.95597839|
|     496545|40.77680206|-73.95554352|
|     300123|40.76207352|-73.98092651|
|    1216556|40.75581741|  -73.961586|
|    1120793|40.75146103|-73.98009491|
|     163818|40.75507355|-73.99047852|
|       8554|40.70281219|-74.01144409|
|    1682356|40.77541351|-73.95649719|
|     543786|40.76676178|-73.92095947|
+-----------+-----------+------------+
only showing top 20 rows



In [22]:
request_df.show(5)
trip_df.show(5)

+----------+------------+------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+
|request_id|passenger_id|pickup_location_id|dropoff_location_id|       request_time|        accept_time|    processing_date|record_modified_date|         record_hash|
+----------+------------+------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+
|    447450|      224189|            376530|             338646|2015-01-04 15:28:00|2015-01-04 15:30:00|2024-10-13 19:45:15|2024-10-08 09:47:...|02018d1c2a52d91dd...|
|    433893|      217445|            562731|             567647|2015-01-29 12:34:00|2015-01-29 12:44:00|2024-10-13 19:45:15|2024-10-08 09:47:...|eef0dbb103d3fbc24...|
|    178990|       88035|            544948|             513195|2015-01-30 17:50:00|2015-01-30 17:51:00|2024-10-13 19:45:15|2024-10-08 09:47:...|536eeb49f7dca596d...

In [23]:
# Join the DataFrames
fact_request = request_df.join(trip_df, request_df["request_id"] == trip_df["request_id"], "left") \
    .join(dim_payment, trip_df["payment_id"] == dim_payment["s_payment"], "left") \
    .select(
        trip_df["driver_id"],
        request_df["passenger_id"],
        request_df["pickup_location_id"],
        request_df["dropoff_location_id"],
        date_format(request_df["request_time"], 'yyyyMMdd').alias("request_datekey"),  # Renamed and formatted
        date_format(request_df["accept_time"], 'yyyyMMdd').alias("accept_datekey"),    # Renamed and formatted
        date_format(trip_df["trip_start_time"], 'yyyyMMdd').alias("trip_start_datekey"),   # Renamed
        date_format(trip_df["trip_end_time"], 'yyyyMMdd').alias("trip_end_datekey"),       # Renamed
        dim_payment["s_payment"],
        trip_df["trip_distance"],
        trip_df["base_fare"],
        trip_df["extra_fare"],
        trip_df["mta_tax"],
        trip_df["tip_amount"],
        trip_df["tolls_amount"],
        trip_df["improvement_surcharge"]
    )


# Show the resulting DataFrame
fact_request.show(10)

+---------+------------+------------------+-------------------+---------------+--------------+------------------+----------------+---------+-------------+---------+----------+-------+----------+------------+---------------------+
|driver_id|passenger_id|pickup_location_id|dropoff_location_id|request_datekey|accept_datekey|trip_start_datekey|trip_end_datekey|s_payment|trip_distance|base_fare|extra_fare|mta_tax|tip_amount|tolls_amount|improvement_surcharge|
+---------+------------+------------------+-------------------+---------------+--------------+------------------+----------------+---------+-------------+---------+----------+-------+----------+------------+---------------------+
|   951731|         801|            384446|             313498|       20150102|      20150102|          20150102|        20150102|        6|          0.5|      4.0|       1.0|    0.5|       0.0|         0.0|                  0.0|
|  1006899|       17348|            402794|             289862|       20150118| 

In [46]:
dataframes = [dim_calendar, dim_location, dim_payment, dim_user, fact_request]
# Names of the original DataFrames
dataframe_names = ["dim_calendar", "dim_location", "dim_payment", "dim_user", "fct_request"]

In [47]:
processing_date = date_trunc('second', current_timestamp())

for i, df in enumerate(dataframes):
    # Apply column name conversion
    df = df.withColumn("processing_date", processing_date)

    # Reassign the modified DataFrame back to the original global variable
    globals()[dataframe_names[i]] = df
    
    # Show the first 5 rows of the updated DataFrame
trip_df.show()

+-------+----------+---------+----------+----------+-------------------+-------------------+-------------+---------+----------+-------+----------+------------+---------------------+-------------------+--------------------+--------------------+
|trip_id|request_id|driver_id|vehicle_id|payment_id|    trip_start_time|      trip_end_time|trip_distance|base_fare|extra_fare|mta_tax|tip_amount|tolls_amount|improvement_surcharge|    processing_date|record_modified_date|         record_hash|
+-------+----------+---------+----------+----------+-------------------+-------------------+-------------+---------+----------+-------+----------+------------+---------------------+-------------------+--------------------+--------------------+
|  52273|    387479|  1133778|    241226|         6|2015-01-06 08:17:00|2015-01-06 08:21:00|         0.57|      4.5|       0.0|    0.5|      1.35|         0.0|                  0.3|2024-10-13 19:45:21|2024-10-08 09:47:...|55eac87a0175d172d...|
|  52513|    304466|  10

In [48]:

# List of DataFrames, locations, and hash columns with an attribute for slowly changing dimension (SCD)
tables_info = [
    {"df": dim_user, "location": f"{gold_location}/DIM_USER", "hash_columns": ["user_id", "full_name"], "is_scd": True},
    {"df": dim_calendar, "location": f"{gold_location}/DIM_CALENDAR", "hash_columns": ["date_key"], "is_scd": False},
    {"df": dim_location, "location": f"{gold_location}/DIM_LOCATION", "hash_columns": ["location_id"], "is_scd": False},
    {"df": dim_payment, "location": f"{gold_location}/DIM_PAYMENT", "hash_columns": ["s_payment"], "is_scd": False},
    {"df": fact_request, "location": f"{gold_location}/FCT_REQUEST", "hash_columns": ["driver_id", "passenger_id", "request_datekey"], "is_scd": False}
]

def deduplicate_source_df(source_df: DataFrame, hash_columns: list) -> DataFrame:
    """
    Deduplicates the source DataFrame based on the hash columns.
    
    Args:
        source_df (DataFrame): The source DataFrame to be deduplicated.
        hash_columns (list): List of columns to be used for deduplication.
        
    Returns:
        DataFrame: The deduplicated DataFrame.
    """
    return source_df.dropDuplicates(hash_columns)

# Function to generate a hash column for specified columns in a DataFrame
def generate_hash_column(df: DataFrame, columns: list, hash_column_name: str = "record_hash") -> DataFrame:
    """
    Generates a hash column for specified columns in the DataFrame using SHA-256.
    """
    return df.withColumn(hash_column_name, sha2(concat_ws("||", *columns), 256))

# Function to build merge condition using hash
def build_merge_condition_with_hash(target_alias: str, source_alias: str, hash_column: str) -> str:
    """
    Builds a merge condition string using the hash column.
    """
    return f"{target_alias}.{hash_column} = {source_alias}.{hash_column}"

# Function to merge Delta tables based on SCD flag
def merge_delta_table(source_df: DataFrame, delta_location: str, hash_columns: list, is_scd: bool):
    """
    Performs merge for slowly changing dimension (SCD) tables. If is_scd is False, data is simply overwritten.
    """
    # Generate hash column for the source DataFrame
    source_df_hashed = generate_hash_column(source_df, hash_columns)
    
    # If the table is an SCD, perform a merge operation
    if is_scd:
        try:
            # Check if the Delta table exists at the specified location
            if DeltaTable.isDeltaTable(spark, delta_location):
                print(f"Delta table found at {delta_location}. Proceeding with merge...")
                delta_table = DeltaTable.forPath(spark, delta_location)
                
                # Alias for target and source
                target_alias = "target"
                source_alias = "src"
                
                # Generate hash column for the target Delta table
                delta_table_df = spark.read.format("delta").load(delta_location)
                delta_table_hashed = generate_hash_column(delta_table_df, hash_columns)
                
                # Build the merge condition using the hash column
                merge_condition = build_merge_condition_with_hash(target_alias, source_alias, "record_hash")
                
                # Perform the merge operation
                delta_table.alias(target_alias).merge(
                    source=source_df_hashed.alias(source_alias),
                    condition=merge_condition
                ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
                
                print(f"Merge completed for {delta_location}.")
            else:
                # If Delta table doesn't exist, write the DataFrame to that location
                print(f"No Delta table found at {delta_location}. Writing new DataFrame...")
                source_df_hashed.write.mode("overwrite").format("delta").save(delta_location)
                print(f"DataFrame written to {delta_location}.")
        except Exception as e:
            print(f"Error processing table at {delta_location}: {e}")
    else:
        # For non-SCD tables, just overwrite the data
        print(f"{delta_location} is not an SCD table. Overwriting...")
        source_df_hashed.write.mode("overwrite").format("delta").save(delta_location)
        print(f"DataFrame overwritten at {delta_location}.")

In [49]:
# Loop through each table and perform the operations
for table_info in tables_info:
    # Print the schema of the source DataFrame
    print("Schema of source DataFrame:")
    table_info["df"].printSchema()  # Ensure we are printing the schema of the specific DataFrame

    # Deduplicate the source DataFrame based on hash columns
    df = deduplicate_source_df(table_info["df"], table_info["hash_columns"])
    
    # Prepare the Delta table location and SCD flag
    delta_location = table_info["location"]
    hash_columns = table_info["hash_columns"]
    is_scd = table_info["is_scd"]  # Check if it's a slowly changing dimension

    # Perform merge or overwrite based on is_scd flag
    merge_delta_table(df, delta_location, hash_columns, is_scd)

Schema of source DataFrame:
root
 |-- user_id: integer (nullable = true)
 |-- full_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- vehicle_id: string (nullable = false)
 |-- vehicle_make: string (nullable = false)
 |-- vehicle_model: string (nullable = false)
 |-- vehicle_year: string (nullable = false)
 |-- vehicle_color: string (nullable = false)
 |-- vehicle_license_plate: string (nullable = false)
 |-- processing_date: timestamp (nullable = true)

Delta table found at e:\DEPIfINALpROJECT\DATABASEcSV/gold/DIM_USER. Proceeding with merge...
Merge completed for e:\DEPIfINALpROJECT\DATABASEcSV/gold/DIM_USER.
Schema of source DataFrame:
root
 |-- date_key: string (nullable = false)
 |-- full_date: date (nullable = false)
 |-- day: integer (nullable = false)
 |-- day_name: string (nullable = false)
 |-- day_of_week: integer (nullable = false)
 |-- week_of_year: integer (nullable = false)
 |-- month: integer (nullable = 

In [50]:
spark.read.format("delta").load(f"{gold_location}/DIM_USER").orderBy("full_name").show()

+-------+------------+--------------------+--------------+----------+------------+-------------+------------+-------------+---------------------+-------------------+--------------------+
|user_id|   full_name|               email|  phone_number|vehicle_id|vehicle_make|vehicle_model|vehicle_year|vehicle_color|vehicle_license_plate|    processing_date|         record_hash|
+-------+------------+--------------------+--------------+----------+------------+-------------+------------+-------------+---------------------+-------------------+--------------------+
|   NULL|    --------|               -----|   -----------| Passenger|   Passenger|    Passenger|   Passenger|    Passenger|            Passenger|2024-10-13 20:06:57|370d9b71060be968d...|
| 768128|Aaron Abbott|angela67@example.com|(453) 832-3608|    168434|     Hyundai|     Santa Fe|        2006|       Silver|              3BW 116|2024-10-13 20:06:57|1ec48e73a3376b697...|
| 762535|Aaron Abbott|angela67@example.com|(453) 832-3608|    168

In [51]:
# After your write_to_delta function
for table in dataframe_names:
    
    print(f"Creating external table: {table}")
    # Creating schema and external table in Spark SQL
    spark.sql("CREATE SCHEMA IF NOT EXISTS gold")
    # Set the current database to 'gold'
    spark.sql("USE gold")
    delta_table_path = f"{gold_location}\\{table}"
    spark.sql(f"CREATE EXTERNAL TABLE IF NOT EXISTS {table} USING DELTA LOCATION r'{table}'")
    #spark.sql(f"SELECT * FROM {table_name}").show()

Creating external table: dim_calendar
Creating external table: dim_location
Creating external table: dim_payment
Creating external table: dim_user
Creating external table: fct_request


In [52]:
from delta.tables import DeltaTable

# Dictionary to store the DeltaTable objects
delta_tables = {}

for table in dataframe_names:
    # Define the table location
    table_location = f"{gold_location}/{table}"
    print(table_location)
    # Store the DeltaTable object in the dictionary using the file name as the key
    delta_tables[table] = DeltaTable.forPath(spark, table_location)  # Load table from path

    # Check if the table has been vacuumed in the last 30 days
    if delta_tables[table].history(30).filter("operation = 'VACUUM START'").count() == 0:
        # Optimize the table for better query performance
        delta_tables[table].optimize()
        # Perform vacuum operation (default is to keep data for 7 days)
        delta_tables[table].vacuum()


e:\DEPIfINALpROJECT\DATABASEcSV/gold/dim_calendar
e:\DEPIfINALpROJECT\DATABASEcSV/gold/dim_location
e:\DEPIfINALpROJECT\DATABASEcSV/gold/dim_payment
e:\DEPIfINALpROJECT\DATABASEcSV/gold/dim_user
e:\DEPIfINALpROJECT\DATABASEcSV/gold/fct_request


In [73]:
spark.read.format("delta").load(f"{gold_location}/FCT_REQUEST").count()


584871

In [68]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, "gold\\FCT_REQUEST")
fullHistoryDF = deltaTable.history()
fullHistoryDF.show(truncate=False)

+-------+-----------------------+------+--------+------------+--------------------------------------------------------------------+----+--------+---------+-----------+-----------------+-------------+---------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation   |operationParameters                                                 |job |notebook|clusterId|readVersion|isolationLevel   |isBlindAppend|operationMetrics                                                     |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+------------+--------------------------------------------------------------------+----+--------+---------+-----------+-----------------+-------------+---------------------------------------------------------------------+------------+-----------------------------------+
|4      |2024-10-13 20:07:59.897|NULL  |NUL

In [53]:
# Join the DataFrames with aliases for clarity
"""
fact_request_with_user_payment = fact_request.alias("fr") \
    .join(dim_user.alias("du"), col("fr.driver_id") == col("du.user_id"), "left") \
    .join(dim_user.alias("pu"), col("fr.passenger_id") == col("pu.user_id"), "left") \
    .join(dim_payment.alias("dp"), col("fr.s_payment") == col("dp.s_payment"), "left") \
    .select(
        col("fr.driver_id"),
        col("fr.passenger_id"),
        col("fr.pickup_location_id"),
        col("fr.dropoff_location_id"),
        col("fr.request_datekey"),
        col("fr.accept_datekey"),
        col("fr.trip_start_datekey"),
        col("fr.trip_end_datekey"),
        col("fr.s_payment"),
        col("fr.trip_distance"),
        col("fr.base_fare"),
        col("fr.extra_fare"),
        col("fr.mta_tax"),
        col("fr.tip_amount"),
        col("fr.tolls_amount"),
        col("fr.improvement_surcharge"),
        col("du.full_name").alias("driver_name"),
        col("du.email").alias("driver_email"),
        col("du.phone_number").alias("driver_phone"),
        col("du.vehicle_id").alias("driver_vehicle_id"),
        col("du.vehicle_make").alias("driver_vehicle_make"),
        col("du.vehicle_model").alias("driver_vehicle_model"),
        col("du.vehicle_year").alias("driver_vehicle_year"),
        col("du.vehicle_color").alias("driver_vehicle_color"),
        col("du.vehicle_license_plate").alias("driver_vehicle_license_plate"),
        col("pu.full_name").alias("passenger_name"),
        col("pu.email").alias("passenger_email"),
        col("pu.phone_number").alias("passenger_phone"),
        col("dp.method").alias("payment_method"),
        col("dp.status").alias("payment_status")
    )

# Show the resulting DataFrame
fact_request_with_user_payment.orderBy("driver_name").show(10)
"""


'\nfact_request_with_user_payment = fact_request.alias("fr")     .join(dim_user.alias("du"), col("fr.driver_id") == col("du.user_id"), "left")     .join(dim_user.alias("pu"), col("fr.passenger_id") == col("pu.user_id"), "left")     .join(dim_payment.alias("dp"), col("fr.s_payment") == col("dp.s_payment"), "left")     .select(\n        col("fr.driver_id"),\n        col("fr.passenger_id"),\n        col("fr.pickup_location_id"),\n        col("fr.dropoff_location_id"),\n        col("fr.request_datekey"),\n        col("fr.accept_datekey"),\n        col("fr.trip_start_datekey"),\n        col("fr.trip_end_datekey"),\n        col("fr.s_payment"),\n        col("fr.trip_distance"),\n        col("fr.base_fare"),\n        col("fr.extra_fare"),\n        col("fr.mta_tax"),\n        col("fr.tip_amount"),\n        col("fr.tolls_amount"),\n        col("fr.improvement_surcharge"),\n        col("du.full_name").alias("driver_name"),\n        col("du.email").alias("driver_email"),\n        col("du.phone_nu